#importing the libraries


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

#Importing the dataset

In [ ]:
rating_df = pd.read_csv('ratings.csv',usecols=['userId','movieId','rating'])
movie_df= pd.read_csv('movies.csv',usecols=['movieId','title'])

In [ ]:
rating_df.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [ ]:
movie_df.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


#Split the training and testing dataset

In [ ]:
from sklearn.model_selection import train_test_split
train_rating_df,test_rating_df = train_test_split(rating_df,test_size=0.2,random_state=1)

In [ ]:
train_rating_df.head()

,userId,movieId,rating
78834,489,3827,2.0
7523,51,1639,5.0
66122,425,5349,3.0
69250,448,3793,4.0
95606,600,3821,1.0


In [ ]:
test_rating_df.head()

,userId,movieId,rating
32974,225,380,3.0
83568,533,103688,5.0
19537,125,149902,2.0
38287,263,1183,5.0
59543,387,1982,3.0


#Create an user-item interaction pivot table

In [ ]:
data_table = rating_df.pivot_table(index='movieId',columns='userId' , values='rating')
data_table =data_table.fillna(0)

In [ ]:
data_table.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
train_table = train_rating_df.pivot_table(index='movieId',columns='userId' , values='rating')
train_table =train_table.fillna(0)

In [ ]:
train_table.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,0.0,2.5,0.0,0.0,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


#Getting global average values

#Create a sparse matrix

In [ ]:
from scipy.sparse import csr_matrix
train_matrix = csr_matrix((train_rating_df.rating.values, (train_rating_df.userId.values, train_rating_df.movieId.values)))
test_matrix = csr_matrix((test_rating_df.rating.values, (test_rating_df.userId.values, test_rating_df.movieId.values)))

#Getting the global value of the whole matrix

In [ ]:
train_average=dict()
train_average['global']=train_matrix.sum() / train_matrix.count_nonzero()
print(train_average['global'])

3.502237566321218


In [ ]:
test_average=dict()
test_average['global']=test_matrix.sum() / test_matrix.count_nonzero()
print(test_average['global'])

3.498834787782626


#Getting each user and each movie average rating

In [ ]:
def get_average_rating(sparse_matrix,is_user):
  ax=1 if is_user else 0
  rating_sum= sparse_matrix.sum(axis=ax).A1
  is_rated = sparse_matrix!=0
  no_of_rating = is_rated.sum(axis=ax).A1
  u,m= sparse_matrix.shape
  average_rating= {i : rating_sum[i]/no_of_rating[i] for i in range (u if is_user else m) if no_of_rating[i]!=0}
  return average_rating

In [ ]:
train_average['user']= get_average_rating(train_matrix,True)
print(train_average['user'])

{1: 4.3841807909604515, 2: 4.0227272727272725, 3: 2.3823529411764706, 4: 3.5681818181818183, 5: 3.6285714285714286, 6: 3.4941634241245136, 7: 3.1141732283464565, 8: 3.5945945945945947, 9: 3.2857142857142856, 10: 3.2242990654205608, 11: 3.8085106382978724, 12: 4.462962962962963, 13: 3.586206896551724, 14: 3.212121212121212, 15: 3.5, 16: 3.759493670886076, 17: 4.202247191011236, 18: 3.713554987212276, 19: 2.618018018018018, 20: 3.6457286432160805, 21: 3.246518105849582, 22: 2.7065217391304346, 23: 3.67, 24: 3.652941176470588, 25: 4.791666666666667, 26: 3.1666666666666665, 27: 3.552380952380952, 28: 3.016304347826087, 29: 4.109375, 30: 4.64, 31: 3.9523809523809526, 32: 3.744186046511628, 33: 3.776, 34: 3.4153846153846152, 35: 4.111111111111111, 36: 2.6979166666666665, 37: 3.933333333333333, 38: 3.152542372881356, 39: 3.8658536585365852, 40: 3.7162162162162162, 41: 3.3035714285714284, 42: 3.616045845272206, 43: 4.574468085106383, 44: 3.380952380952381, 45: 3.892523364485981, 46: 4.05405405

In [ ]:
train_average['movie']= get_average_rating(train_matrix,False)
print(train_average['movie'])

{1: 3.9415204678362574, 2: 3.4310344827586206, 3: 3.2666666666666666, 4: 2.625, 5: 3.0657894736842106, 6: 3.8860759493670884, 7: 3.2386363636363638, 8: 2.875, 9: 3.0714285714285716, 10: 3.5288461538461537, 11: 3.7232142857142856, 12: 2.4375, 13: 3.0, 14: 3.8666666666666667, 15: 2.75, 16: 3.8768115942028984, 17: 3.78, 18: 3.65625, 19: 2.6733333333333333, 20: 2.5833333333333335, 21: 3.5714285714285716, 22: 3.32, 23: 2.9166666666666665, 24: 3.1956521739130435, 25: 3.626865671641791, 26: 3.6875, 27: 3.125, 28: 4.214285714285714, 29: 4.044117647058823, 30: 3.0, 31: 3.171875, 32: 3.962068965517241, 34: 3.6346153846153846, 36: 3.8706896551724137, 38: 3.0, 39: 3.2098765432098766, 40: 5.0, 41: 3.5454545454545454, 42: 3.0, 43: 3.0, 44: 2.554054054054054, 45: 3.173076923076923, 46: 3.0833333333333335, 47: 3.9622641509433962, 48: 3.1228070175438596, 49: 3.0, 50: 4.258064516129032, 52: 3.5, 53: 5.0, 54: 2.3333333333333335, 55: 4.0, 57: 2.9166666666666665, 58: 4.032258064516129, 60: 3.04, 61: 3.5, 6

In [ ]:
test_average['user']= get_average_rating(test_matrix,True)
print(test_average['user'])

{1: 4.3090909090909095, 2: 3.7142857142857144, 3: 2.8, 4: 3.5, 5: 3.6666666666666665, 6: 3.491228070175439, 7: 3.82, 8: 3.5, 9: 3.1818181818181817, 10: 3.4545454545454546, 11: 3.7058823529411766, 12: 4.0, 13: 4.5, 14: 3.8, 15: 3.230769230769231, 16: 3.5789473684210527, 17: 4.25, 18: 3.7972972972972974, 19: 2.5675675675675675, 20: 3.3372093023255816, 21: 3.3214285714285716, 22: 2.111111111111111, 23: 3.5476190476190474, 24: 3.64, 25: 5.0, 26: 3.6666666666666665, 27: 3.533333333333333, 28: 3.036363636363636, 29: 4.264705882352941, 30: 5.0, 31: 3.75, 32: 3.8125, 33: 3.838709677419355, 34: 3.4285714285714284, 35: 4.0, 36: 2.375, 37: 4.666666666666667, 38: 3.4210526315789473, 39: 4.611111111111111, 40: 3.896551724137931, 41: 3.0816326530612246, 42: 3.3736263736263736, 43: 4.45, 44: 3.1666666666666665, 45: 3.8076923076923075, 46: 3.6, 47: 2.8653846153846154, 48: 4.25, 49: 4.5, 50: 2.8947368421052633, 51: 3.7704918032786887, 52: 4.28, 53: 5.0, 54: 3.0, 55: 4.166666666666667, 56: 3.4, 57: 3.60

In [ ]:
test_average['movie']= get_average_rating(test_matrix,False)
print(test_average['movie'])

{1: 3.840909090909091, 2: 3.4347826086956523, 3: 3.2142857142857144, 4: 2.0, 5: 3.090909090909091, 6: 4.1521739130434785, 7: 2.95, 9: 3.5, 10: 3.375, 11: 3.4642857142857144, 12: 2.3333333333333335, 13: 4.0, 14: 3.6666666666666665, 15: 3.8333333333333335, 16: 4.1923076923076925, 17: 3.764705882352941, 18: 3.875, 19: 3.0384615384615383, 20: 2.1666666666666665, 21: 3.210526315789474, 22: 3.0, 23: 3.75, 24: 2.8, 25: 3.611111111111111, 26: 3.2, 27: 5.0, 28: 4.25, 29: 3.75, 31: 3.25, 32: 4.078125, 34: 3.7291666666666665, 36: 3.611111111111111, 38: 1.8333333333333333, 39: 3.5869565217391304, 40: 3.5, 41: 4.125, 43: 3.375, 44: 2.5, 45: 3.9166666666666665, 46: 3.0, 47: 4.0227272727272725, 48: 3.272727272727273, 50: 4.173469387755102, 52: 3.357142857142857, 57: 3.5, 58: 4.0, 60: 3.7777777777777777, 61: 4.0, 62: 3.625, 63: 2.5, 64: 3.0, 65: 2.5714285714285716, 66: 3.0, 68: 5.0, 69: 3.6, 70: 3.772727272727273, 71: 1.75, 72: 3.6666666666666665, 73: 3.5833333333333335, 74: 3.5, 76: 2.0, 79: 3.25, 81

#Create KNN model

In [ ]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(n_neighbors=20,algorithm='brute',metric='cosine')

#Train KNN on training set

In [ ]:
model.fit(train_table)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=20)

#Function to get predictions

In [ ]:
def get_predictions(rating_df,data_table,g_average):
  start = datetime.datetime.now()
  data_user= rating_df['userId']
  data_movie= rating_df['movieId']

  final_data = pd.DataFrame()
  count=0


  for(user,movie) in zip(data_user,data_movie):
    total_score=0
    total_rating=0
    final_score=0
    similar=model.kneighbors(data_table[data_table.index==movie],return_distance=True)
    sim_score=np.array(similar[0])
    sim_id=np.array(similar[1])
    for i in range(1,len(sim_score[0])):
      single_rating=data_table[user].iloc[sim_id[0,i]]
      if single_rating!=0:
        total_rating= total_rating + sim_score[0,i] * single_rating
        total_score= total_score + sim_score[0,i]
    if total_score!=0:
      final_score=total_rating/total_score
    else:
      final_score=(g_average['user'][user]+g_average['movie'][movie])/2
    row= list()
    row.append(user)
    row.append(movie)
    row.append(final_score)
    row_df= pd.DataFrame(row).T
    final_data = pd.concat([final_data,row_df],ignore_index = True)
    count=count+1
    if (count)%1000==0:
      print("Done for {} rows----- {}".format(count, datetime.datetime.now() - start))
  return final_data

In [ ]:
final_train_data=get_predictions(train_rating_df,data_table,train_average)
final_train_data.columns=['userId', 'movieId', 'rating']
final_train_data.to_csv('train_final_data.csv')

Done for 1000 rows----- 0:01:22.402970
Done for 2000 rows----- 0:02:26.373381
Done for 3000 rows----- 0:03:28.315098
Done for 4000 rows----- 0:04:29.366059
Done for 5000 rows----- 0:05:31.472473
Done for 6000 rows----- 0:06:34.158591
Done for 7000 rows----- 0:07:36.077179
Done for 8000 rows----- 0:08:38.194701
Done for 9000 rows----- 0:09:39.024852
Done for 10000 rows----- 0:10:40.581336
Done for 11000 rows----- 0:11:42.632778
Done for 12000 rows----- 0:12:45.241533
Done for 13000 rows----- 0:13:47.236812
Done for 14000 rows----- 0:14:48.661161
Done for 15000 rows----- 0:15:50.232007
Done for 16000 rows----- 0:16:52.217054
Done for 17000 rows----- 0:17:54.127503
Done for 18000 rows----- 0:18:56.138632
Done for 19000 rows----- 0:19:57.584065
Done for 20000 rows----- 0:20:59.091599
Done for 21000 rows----- 0:22:01.708319
Done for 22000 rows----- 0:23:04.299998
Done for 23000 rows----- 0:24:06.579093
Done for 24000 rows----- 0:25:09.021946
Done for 25000 rows----- 0:26:11.478245
Done for 

In [ ]:
final_test_data=get_predictions(test_rating_df,data_table,test_average)
final_test_data.columns=['userId', 'movieId', 'rating']
final_test_data.to_csv('test_final_data.csv')

Done for 1000 rows----- 0:00:59.965412
Done for 2000 rows----- 0:02:01.288799
Done for 3000 rows----- 0:03:02.303575
Done for 4000 rows----- 0:04:03.220724
Done for 5000 rows----- 0:05:03.593091
Done for 6000 rows----- 0:06:04.825984
Done for 7000 rows----- 0:07:06.456202
Done for 8000 rows----- 0:08:07.915971
Done for 9000 rows----- 0:09:08.171347
Done for 10000 rows----- 0:10:09.510778
Done for 11000 rows----- 0:11:10.976039
Done for 12000 rows----- 0:12:12.458625
Done for 13000 rows----- 0:13:12.526820
Done for 14000 rows----- 0:14:14.336423
Done for 15000 rows----- 0:15:15.822496
Done for 16000 rows----- 0:16:29.855892
Done for 17000 rows----- 0:17:47.228326
Done for 18000 rows----- 0:18:54.886732
Done for 19000 rows----- 0:19:56.332843
Done for 20000 rows----- 0:20:56.669789


In [ ]:
print(final_train_data)

       userId  movieId    rating
0       489.0   3827.0  3.499373
1        51.0   1639.0  4.000000
2       425.0   5349.0  3.558138
3       448.0   3793.0  4.000000
4       600.0   3821.0  4.500000
...       ...      ...       ...
80663   322.0   4226.0  3.758067
80664   606.0   4973.0  4.000000
80665    34.0    318.0  3.909081
80666   483.0   8644.0  3.504155
80667   607.0   1213.0  5.000000

[80668 rows x 3 columns]


In [ ]:
print(final_test_data)

       userId   movieId    rating
0       225.0     380.0  3.716028
1       533.0  103688.0  4.354167
2       125.0  149902.0  2.911765
3       263.0    1183.0  4.508523
4       387.0    1982.0  3.757337
...       ...       ...       ...
20163   381.0   69436.0  3.472846
20164   253.0    5991.0  3.916667
20165   115.0    3052.0  3.512238
20166   217.0    2605.0  2.483278
20167   414.0    4900.0  3.464135

[20168 rows x 3 columns]


In [ ]:
def get_error_metrics(y_true, y_pred):
    rmse = np.sqrt(np.mean([ (y_true[i] - y_pred[i])**2 for i in range(len(y_pred)) ]))
    mape = np.mean(np.abs( (y_true - y_pred)/y_true )) * 100
    return rmse, mape

#Testing Training data

In [ ]:
y_test=train_rating_df['rating']
print(y_test.values)

[2. 5. 3. ... 4. 4. 4.]


In [ ]:
y_pred=final_train_data['rating']
print(y_pred.values)

[3.49937348 4.         3.55813775 ... 3.9090812  3.50415495 5.        ]


In [ ]:
rsme_train,mape_train = get_error_metrics(y_test.values,y_pred.values)
train_results = {
    'rsme':rsme_train,
    'mape':mape_train
}

In [ ]:
print(train_results)

{'rsme': 1.0304691212339059, 'mape': 33.44024611963449}


#Testing on test set

In [ ]:
y_test=test_rating_df['rating']
print(y_test.values)

[3. 5. 2. ... 3. 2. 4.]


In [ ]:
y_pred=final_test_data['rating']
print(y_pred.values)

[3.71602787 4.35416667 2.91176471 ... 3.51223776 2.48327767 3.4641349 ]


In [ ]:
rsme_test,mape_test = get_error_metrics(y_test.values,y_pred.values)
test_results = {
    'rsme':rsme_test,
    'mape':mape_test
}

In [ ]:
print(test_results)

{'rsme': 1.022684442842138, 'mape': 33.05214710924582}


#Reccomendation system

#Input the user to recommend movies to

In [ ]:
selected_id= input(' Enter the userId of the user to recommend to : ')
selected_id = int(selected_id)

 Enter the userId of the user to recommend to : 5


#Getting the selected user's rated movies

In [ ]:
selected_user= data_table[selected_id]
selected_user = selected_user.loc[selected_user!=0]
selected_user.head()

movieId
1     4.0
21    4.0
34    4.0
36    4.0
39    3.0
Name: 5, dtype: float64

#List of movies that rated by at least 50 users

In [ ]:
stats= rating_df.groupby('movieId').agg({'rating':[np.size]})
stats.head(10 )
stats= stats[stats['rating']['size']>=50]

In [ ]:
stats.head()

,rating
,size
movieId,
1,215
2,110
3,52
6,102
7,54


#Function to get recommendations and predict ratings that user would give to each movie

In [ ]:
def get_recommendation():
  similar_candidates_rating= pd.Series(dtype='float64')
  similar_candidates_score= pd.Series(dtype='float64')
  for i in range(0,len(selected_user.index)):
    similar=model.kneighbors([data_table.loc[selected_user.index[i]]],return_distance=True)
    sim_score=similar[0]
    sim_id=similar[1]
    sim_id=np.array(list(map(lambda x: data_table.iloc[x].name,sim_id[0])))
    similar=pd.Series(data=sim_score[0],index=sim_id)
    similar=similar[similar!=0]
    similar=similar[similar.index.isin(stats.index)]
    similar_candidates_score=pd.concat([similar_candidates_score,similar])
    similar=similar.map(lambda x: x*selected_user.values[i])
    similar_candidates_rating = pd.concat([similar_candidates_rating,similar])
  return similar_candidates_rating,similar_candidates_score

In [ ]:
similar_candidates_rating,similar_candidates_score= get_recommendation()

#Factoring the recommendation score to the scale of 1-5

In [ ]:
similar_candidates_rating.sort_values(inplace=True,ascending=False)
similar_candidates_score.sort_values(inplace=True,ascending=False)
filtered_candidates_rating_sum= similar_candidates_rating.groupby(similar_candidates_rating.index).sum()
filtered_candidates_score_sum= similar_candidates_score.groupby(similar_candidates_score.index).sum()

In [ ]:
similar_movies=filtered_candidates_rating_sum.index
pred_rating= pd.Series(dtype='float64',index=similar_movies)
for i in range(0,len(similar_movies)):
  pred_rating[similar_movies[i]]= filtered_candidates_rating_sum[similar_movies[i]]/filtered_candidates_score_sum[similar_movies[i]]

#Making Recommendations to the selected user

#The user's orignal highest rated films

In [ ]:
selected_user.sort_values(inplace=True,ascending=False)
selected_user_df = pd.DataFrame(selected_user).reset_index()
selected_user_df.columns = ['movieId', 'predicted_rating']
selected_user_df = pd.merge(selected_user_df,movie_df)
selected_user_df.head(10)

,movieId,predicted_rating,title
0,590,5.0,Dances with Wolves (1990)
1,475,5.0,In the Name of the Father (1993)
2,527,5.0,Schindler's List (1993)
3,58,5.0,"Postman, The (Postino, Il) (1994)"
4,596,5.0,Pinocchio (1940)
5,595,5.0,Beauty and the Beast (1991)
6,247,5.0,Heavenly Creatures (1994)
7,594,5.0,Snow White and the Seven Dwarfs (1937)
8,296,5.0,Pulp Fiction (1994)
9,290,5.0,Once Were Warriors (1994)


#Recommendations made by KNN

In [ ]:
print("List of recommendations for user {}".format(selected_id))
pred_rating.sort_values(inplace=True,ascending=False)
pred_rating_df = pd.DataFrame(pred_rating).reset_index()
pred_rating_df.columns = ['movieId', 'predicted_rating']
final_pred_df = pd.merge(pred_rating_df,movie_df)
final_pred_df.head(10)

List of recommendations for user 5


,movieId,predicted_rating,title
0,2100,5.000000,Splash (1984)
1,1721,5.000000,Titanic (1997)
2,733,5.000000,"Rock, The (1996)"
3,594,5.000000,Snow White and the Seven Dwarfs (1937)
4,300,5.000000,Quiz Show (1994)
5,48,5.000000,Pocahontas (1995)
6,353,4.499615,"Crow, The (1994)"
7,587,4.316220,Ghost (1990)
8,296,4.051312,Pulp Fiction (1994)
9,1148,4.043734,Wallace & Gromit: The Wrong Trousers (1993)
